In [349]:
# Had issues installing pyreadstat so used magic command in notebook instead
# %pip install pyreadstat

import glob
import pyreadstat
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from pathlib import Path

In [350]:
# Establish path to data folder
DATA_DIR = Path('/Volumes/WRKGRP/STD-FSW-BSI-SD-Movement_Tracking/dsp/data')

# Tracking data

Tracking data is not processed here. The files are only loaded to collect TrackLab IDs, school and class numbers, and dates. This information is added to the merged file export (see #Exports).

## Determine available data

Only 2023 tracking data has been loaded since we do not have the survey responses to 2022 data.

In [351]:
filename = 'tracking-data-all'
path = DATA_DIR / '02_interim' / f"{filename}.csv"

summary_filename = 'tracking-data-summary'
summary_path = DATA_DIR / '02_interim' / f"{summary_filename}.csv"

# If files already exist, load them
if path.exists() and summary_path.exists():
    print(f'Loaded file: {filename}.csv')
    tracking_data = pd.read_csv(path).astype(str)
    tracking_data['date'] = pd.to_datetime(tracking_data['date'])
    print(f'Loaded file: {summary_filename}.csv')
    tracking_summary = pd.read_csv(summary_path).astype(str)
    tracking_summary['date'] = pd.to_datetime(tracking_summary['date'])    
# Else, create them from raw files
else:
    # Get 2023 school/class info from tracking data subfolder names
    folder = DATA_DIR / '01_tracking' / '2023'
    subfolders = [f.name for f in folder.iterdir() if f.is_dir()]
    
    # Initialize list to store all tracking data entries
    all_data = []
    
    # Process each subfolder
    for subfolder in subfolders:
        date, school_num = subfolder.split('_s')
        school_num, class_num = school_num.split('_c')
        
        # Get the full path to the subfolder
        subfolder_path = folder / subfolder
        
        # Find all CSV files in the subfolder
        csv_files = list(subfolder_path.glob('*.csv'))
        
        if csv_files:
            # Process each CSV file in the subfolder
            for csv_file in csv_files:
                file_name = csv_file.name
                
                # Extract tracklab_id from filename by splitting on '] ' and taking the part after
                if '] ' in file_name:
                    tracklab_id = file_name.split('] ', 1)[1]
                    # Remove .csv extension if present
                    tracklab_id = tracklab_id.replace('.csv', '')
                else:
                    # Handle case where the expected delimiter isn't found
                    tracklab_id = file_name.replace('.csv', '')
                
                # Add entry for this specific CSV file
                all_data.append([date, school_num, class_num, tracklab_id])
        else:
            # If no CSV files found, still add the folder info without tracklab_id
            all_data.append([date, school_num, class_num, None])
    
    # Create df with date, school, class, and tracklab_id
    tracking_data = pd.DataFrame(all_data, columns=['date', 'school', 'class', 'tracklab_id'])
    tracking_data = tracking_data.astype(str)
    tracking_data['date'] = pd.to_datetime(tracking_data['date'], format='%Y_%m_%d')
    
    # Replace school 1 with 44 to align data
    tracking_data.loc[tracking_data['school']=='1', 'school'] = '44'
    
    # Create summary df of unique date, school, and class combinations
    tracking_summary= tracking_data[['date', 'school', 'class']].drop_duplicates().reset_index(drop=True)
    
    # Save dfs as csv
    summary_filename = 'tracking-data-summary'
    summary_path = DATA_DIR / '02_interim' / f"{summary_filename}.csv"
    summary_path.parent.mkdir(parents=True, exist_ok=True)
    tracking_summary.to_csv(summary_path, index=False)
    print("Summary data saved to", summary_path)
    
    path.parent.mkdir(parents=True, exist_ok=True)
    tracking_data.to_csv(path, index=False)
    print("All data saved to", path)

tracking_data.groupby(['date', 'school', 'class'])['tracklab_id'].nunique().reset_index(name='N tracklab_id')

Loaded file: tracking-data-all.csv
Loaded file: tracking-data-summary.csv


,date,school,class,N tracklab_id
0,2023-04-11,42,102,16
1,2023-05-11,43,103,30
2,2023-05-23,46,107,20
3,2023-05-24,47,108,15
4,2023-05-31,44,104,22
5,2023-06-08,45,105,22
6,2023-06-09,45,106,16


## Inspect TrackLab files

### Raw file load
Just looking at one file to see how it's behaving because min-max numbers looked crazy.

In [352]:
# # Establish data folder and inspect subfolders
# filepath = DATA_DIR / '01_tracking' / '2023'
# subfolders = [p for p in filepath.iterdir() if p.is_dir()]
# # print(subfolders)
# 
# # Choose one subfolder (tracking day) for random check
# subfolder_path = filepath / '2023_04_11_s42_c102'
# csv_files = list(subfolder_path.glob('*.csv'))
# print(f"\nInspecting {subfolder_path}")
# print(f"N of TrackLab files in subfolder: {len(csv_files)}")
# 
# # Load all csv files separately into one df, then concatenate
# # Takes almost 30sec on my laptop...
# tracklab = [pd.read_csv(file, sep=';') for file in csv_files]
# print(f"N of TrackLab files loaded into dataframe: {len(tracklab)}")
# tracklab = pd.concat(tracklab, ignore_index=True) 

In [353]:
# Search for example of crazy value like 940721624896047 in this df -> literal int did not exist, checking for decimal
# tracklab[tracklab['X']>=9.4].sort_values(by='X').head()  # Think I found it (idx 102265)

In [354]:
# # Check min-max XYZ values
# pivot = pd.pivot_table(
#     tracklab,
#     index='TagId',
#     values=['X', 'Y', 'Z'],
#     aggfunc={'X': ['min', 'mean', 'max'], 'Y': ['min', 'max'], 'Z': ['min', 'max']}
# )
# 
# # Rename cols for clarity
# pivot.columns = [f"{col}_{func}" for col, func in pivot.columns]
# 
# pivot

### Oliver's method

This code brings down any outliers to value<10, probably should not be doing that.

In [355]:
# oliver = [pd.read_csv(file, sep=';') for file in csv_files]

In [356]:
# def fix_coordinates(value):
# 
#     #fixes issues of values like 940721624896047
#     while abs(value) > 10:
#         value /= 10  # moves the decimal one to the left
# 
#     return value
# 
# #just a sanity check, will remove in final version
# for df in oliver:
#     for col in ['X', 'Y', 'Z']:
#         df[col] = df[col].apply(fix_coordinates)
#     print(df['TagId'].unique())
#     print(df['X'].max())
#     print(df['X'].min())

In [357]:
# oliver = pd.concat(oliver, ignore_index=True)
# # Check min-max XYZ values
# pivot = pd.pivot_table(
#     oliver,
#     index='TagId',
#     values=['X', 'Y', 'Z'],
#     aggfunc={'X': ['min', 'mean', 'max'], 'Y': ['min', 'max'], 'Z': ['min', 'max']}
# )
# 
# # Rename cols for clarity
# pivot.columns = [f"{col}_{func}" for col, func in pivot.columns]
# 
# pivot

## Animate movement trajectories

**14-04-2025:** An attempt. Not successful because timestamps need to be in sequence, and they are currently not.

In [358]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from IPython.display import HTML
# 
# # Load the CSV file (adjust the path accordingly)
# # file_path = "your_filename.csv"  # Replace with the correct file path
# # df = pd.read_csv(file_path, delimiter=';')
# 
# # Filter data for a single TagId
# tag_id = tracklab['TagId'].unique()[0]
# tag_data = tracklab[tracklab['TagId'] == tag_id].copy()
# 
# # Convert TimeStamp to datetime and sort
# tag_data['TimeStamp'] = pd.to_datetime(tag_data['TimeStamp'], format="%m/%d/%Y %H:%M:%S.%f")
# tag_data.sort_values('TimeStamp', inplace=True)
# 
# # Set up the plot
# fig, ax = plt.subplots(figsize=(8, 6))
# ax.set_xlim(tag_data['X'].min() - 1, tag_data['X'].max() + 1)
# ax.set_ylim(tag_data['Y'].min() - 1, tag_data['Y'].max() + 1)
# ax.set_title(f'Movement Path Animation for Tag {tag_id}')
# ax.set_xlabel('X (meters)')
# ax.set_ylabel('Y (meters)')
# 
# # Line and moving point
# line, = ax.plot([], [], lw=2)
# point, = ax.plot([], [], 'ro')
# 
# # Initialization function
# def init():
#     line.set_data([], [])
#     point.set_data([], [])
#     return line, point
# 
# # Animation function
# def animate(i):
#     x = tag_data['X'].iloc[:i]
#     y = tag_data['Y'].iloc[:i]
#     line.set_data(x, y)
#     if i > 0:
#         point.set_data(x.iloc[-1], y.iloc[-1])
#     return line, point
# 
# # Create the animation
# ani = animation.FuncAnimation(
#     fig, animate, init_func=init,
#     frames=len(tag_data), interval=20, blit=True
# )
# 
# # Display the animation inline
# HTML(ani.to_jshtml())


# Keyfiles

## TrackLabID keyfile

One CSV file connects TrackLabID numbers to tag numbers. Each tag was placed on a student, and connected to them using an ID number noted down in separate keyfiles (processed below).

In [359]:
filename = 'keyfile_tracklab_id'
path = DATA_DIR / '02_interim' / f"{filename}.csv"

if path.exists():
    # Load formatted keyfile
    print(f'Loaded file: {filename}.csv')
    keyfile_tagID = pd.read_csv(path).astype(str)
else:
    # Load and format raw keyfile + save 
    path_raw = DATA_DIR / 'keyfiles' / 'Keyfile_csv.csv'
    keyfile_tagID = pd.read_csv(path_raw, delimiter=';')
    
    keyfile_tagID = keyfile_tagID.astype(str)
    keyfile_tagID = keyfile_tagID.rename(columns={'Tagnumber': 'tag_id', 'TrackLabID': 'tracklab_id'})
    
    path.parent.mkdir(parents=True, exist_ok=True)
    keyfile_tagID.to_csv(path, index=False)
    print("Data saved to", path)
    
# Uncomment to view keyfile_tracklab_id
# display(keyfile_tagID)

Loaded file: keyfile_tracklab_id.csv


## Classroom keyfiles

Keyfiles **do not** share the same structure. Columns containing tag numbers are called `tagnummer`, `tagnummer `, `tagnr`, etc. Teachers have not been added to the keyfiles according to an ID number; mostly they are denoted as 'leerkracht' in various columns. 

Columns `sID_survey` and `ID_survey` have been added **manually** from the student response master sheet `TotalData_T1_all_cbs_ethnicity_gender.xlsx` after determining that the 4-digit ID numbers do not match across the individual keyfiles and master sheet, and aligning the data through code proved more time-consuming than copy/pasting.

Columns containing tag numbers have been merged into one column `tagnumber`.

Column `source` has been added to denote source file. 

Column `comment` has been added as a container for any observations made in the classroom.

Teacher have been assigned their school, class, and placeholder ID numbers (9999).  

In [360]:
#todo put this function in a separate file
def process_excel_columns(df):
    
    # Process first two columns
    for col_idx in [0, 1]:  # First and second columns (0-indexed)
        if len(df.columns) > col_idx:  # Check if column exists
            col_name = df.columns[col_idx]
            
            # Extract unique digits from column
            unique_digits = set()
            for val in df.iloc[:, col_idx]:
                try:
                    # Convert to int
                    num = int(float(val))
                    unique_digits.add(num)
                except (ValueError, TypeError):
                    # Skip non-numeric values
                    continue
            
            # If exactly one unique digit found, fill entire column
            if len(unique_digits) == 1:
                digit = unique_digits.pop()
                df.iloc[:, col_idx] = digit
                print(f"Column {col_idx+1} ('{col_name}') filled with value: {digit}")
            else:
                print(f"Column {col_idx+1} ('{col_name}') has {len(unique_digits)} unique digits - no action taken")
    
    return df


### LOAD KEYFILES
folder = DATA_DIR / 'keyfiles'

# Initiate empty df
keyfiles = pd.DataFrame()

print('Loading keyfiles...\n-----')

for file in folder.glob('*.xlsx'):
    try:
        df = pd.read_excel(file, engine='openpyxl')
        df['source'] = file.stem  # Add filename as source in df
        print(f'Loading: {file.name}')
        df = process_excel_columns(df)
        keyfiles = pd.concat([keyfiles, df], ignore_index=True)
        print('-----')
    except:
        # Hidden Excel temp files can mess up the loop
        continue

# Convert all entries to string
keyfiles = keyfiles.apply(lambda x: x.apply(lambda y: str(int(y)) if pd.notna(y) and isinstance(y, (float, int)) else y))


### RENAME COLS & ADD DATES
# Rename columns for alignment
keyfiles = keyfiles.rename(columns={'school ID': 'school', 'klas ID': 'class', 'id': 'person_id', 'subject ID': 'subject_id'})

# Add dates to keyfile by mapping
keyfiles['date'] = np.nan
date_map = tracking_summary.set_index(['school', 'class'])['date'].to_dict()

# # Update 'date' in keyfiles where keys match
keyfiles['date'] = keyfiles.apply(
    lambda row: date_map.get((row['school'], row['class']), row['date']),
    axis=1
)

# Uncomment the lines below to see output
# print('Example keyfile entries:')
# keyfiles.head()

Loading keyfiles...
-----
Loading: keyfile school 45 class 105.xlsx
Column 1 ('school ID') filled with value: 45
Column 2 ('klas ID') filled with value: 105
-----
Loading: keyfile school 1 class 104.xlsx
Column 1 ('school ID') filled with value: 44
Column 2 ('klas ID') filled with value: 104
-----
Loading: keyfile school 41 class 100.xlsx
Column 1 ('school ID') filled with value: 41
Column 2 ('klas ID') filled with value: 100
-----
Loading: keyfile school 41 class 101.xlsx
Column 1 ('school ID') filled with value: 41
Column 2 ('klas ID') filled with value: 101
-----
Loading: keyfile school 42 class 102.xlsx
Column 1 ('school ID') filled with value: 42
Column 2 ('klas ID') filled with value: 102
-----
Loading: keyfile school 43 class 103.xlsx
Column 1 ('school ID') filled with value: 43
Column 2 ('klas ID') filled with value: 103
-----
Loading: keyfile school 46 class 107.xlsx
Column 1 ('school ID') filled with value: 46
Column 2 ('klas ID') filled with value: 107
-----
Loading: keyfile

### Drop irrelevant entries

If school and class were not found in tracking data, drop the corresponding entries from `keyfiles`.  

In [361]:
keyfiles = keyfiles.loc[(keyfiles['school'].isin(tracking_summary['school'])) & (keyfiles['class'].isin(tracking_summary['class']))]

### Comments

In [362]:
# for col in keyfiles.columns[4:]:
#     print(f'Column {col}:\n{keyfiles[col].unique()}\n-----')

cols_keyfiles = keyfiles.columns.tolist() 
idx_source = cols_keyfiles.index('source')
comment_vals = keyfiles[cols_keyfiles[idx_source+1:-1]].stack().groupby(level=0).first()

# Fill keyfiles['comment'] with comment_vals where empty
for idx in keyfiles.index:
    if pd.isna(keyfiles.at[idx, 'comment']) or keyfiles.at[idx, 'comment'] == '':
        if idx in comment_vals.index and not pd.isna(comment_vals[idx]):
            keyfiles.at[idx, 'comment'] = comment_vals[idx]

keyfiles.loc[keyfiles['comment'].astype(str).str.isdigit(), 'comment'] = np.nan
keyfiles.loc[keyfiles['comment']=='-', 'comment'] = np.nan

# Drop old comment columns
keyfiles = keyfiles.drop(columns='Unnamed: 11')

print('Keyfile rows containing comments:')
keyfiles.loc[~keyfiles['comment'].isna(), ['school', 'class', 'subject_id', 'comment']]

Keyfile rows containing comments:


,school,class,subject_id,comment
9,45,105,10,it could be that not tag 12 but 30 was used fo...
112,43,103,4,"niet aanwezig, uit vragenlijst gehaald"
118,43,103,10,Vult vragenlijst niet in
134,43,103,26,niet aanwezig
143,46,107,1,Ja
162,46,107,20,"since student had no written consent, their na..."


### Tag numbers

In [363]:
# Create a new column 'tag_id' that combines all the tag number columns
keyfiles['tag_id'] = np.nan

# Inspect column names
# print(keyfiles.columns)

# Create list of tag number columns
tag_columns = ['tagnummer', 'tagnummer ', 'tagnr.', 'tagnr', 'trackingnnumer']

# Fill NaN values in 'tag_id' with values from other tag columns
for col in tag_columns:
    keyfiles['tag_id'] = keyfiles['tag_id'].fillna(keyfiles[col])

# Drop old tag number columns
keyfiles = keyfiles.drop(columns=tag_columns)

### REPLACE EMPTY TAGNUMBERS WITH 9999
keyfiles.loc[~keyfiles['tag_id'].astype(str).str.isdigit(), 'tag_id'] = '9999'

#### Teacher tags and subject ID

In [364]:
# Fill empty tag_id for teachers with '35' 
for idx, row in keyfiles.iterrows():
    found_in_row = False
    for col in keyfiles.columns:
        cell = row[col]
        if isinstance(cell, str) and 'leerkracht' in cell.lower():
            print(f"Row {idx}: found 'leerkracht' in column '{col}'")
            found_in_row = True
            break  # Stop checking that row
    
    if found_in_row:
        # Check if tag_id is empty or missing
        tag = row['tag_id']
        if pd.isna(tag) or tag == '' or tag is None or tag == '9999':
            keyfiles.at[idx, 'tag_id'] = '35'
            print("'tag_id' was missing or invalid — changed to '35'")
        else:
            print(f"'tag_id' exists: {tag}")
        
        # Always update consent to '1' if found
        keyfiles.at[idx, 'consent'] = '1'
        # Always update subject_id to '9999' if found
        keyfiles.at[idx, 'subject_id'] = '9999'
        
# Drop name columns - not useful anymore
keyfiles = keyfiles.drop(columns=['voornaam', 'achternaam'])

Row 23: found 'leerkracht' in column 'voornaam'
'tag_id' exists: 35
Row 142: found 'leerkracht' in column 'voornaam'
'tag_id' was missing or invalid — changed to '35'
Row 186: found 'leerkracht' in column 'achternaam'
'tag_id' exists: 35
Row 208: found 'leerkracht' in column 'voornaam'
'tag_id' exists: 27


### Consent

Column `consent` contains digits and commentary. Clear commentary such as 'ja' has been transformed into 1, denoting yes. Any entries *not* containing 1 in this column will not be taken into analysis. 

In [365]:
### REPLACE CONSENT ENTRIES WITH DIGITS -> 1 = YES
# Replace positive non-digit consent entries with '1'
keyfiles['consent'] = keyfiles['consent'].replace({'ja': '1'})

### REPLACE NON-CONSENT ENTRIES WITH '9999' -> ENTRIES TO EXCLUDE ARE THEN ['4','5','9']
keyfiles.loc[~keyfiles['consent'].isin(['1','4','5']), 'consent'] = '9999'

# Inspect consent entries
print('Entries in CONSENT:')
print(keyfiles['consent'].unique())


### OLD CODE - TO INSPECT AND PROBABLY DELETE
# sorting_columns = ['subject_id', 'ID_survey', 'tagnumber']
# keyfiles.loc[keyfiles['tagnumber']=='-', 'tagnumber'] = 1111
# keyfiles.loc[~keyfiles['tagnumber'].astype(str).str.isdigit(), 'tagnumber'] = 9999
# # keyfiles.loc[~keyfiles['consent'].astype(str).str.isdigit(), 'consent'] = 5
# keyfiles[sorting_columns] = keyfiles[sorting_columns].fillna(1111).astype(int, errors='raise') 
# keyfiles.sort_values(by=['school', 'class', 'subject_id','consent','tagnumber'])

Entries in CONSENT:
['1' '4' '9999']


## Match TrackLab IDs

In [366]:
# Create mapping dictionary - convert keys to same type as keyfiles['tagnumber']
tracklab_id_map = keyfile_tagID.dropna(subset=['tag_id']).set_index('tag_id')['tracklab_id'].to_dict()

# Map values more efficiently using map() instead of apply
keyfiles['tracklab_id'] = keyfiles['tag_id'].map(tracklab_id_map)

# Sort df
keyfiles.sort_values(by=['school', 'class', 'ID_survey'])

print('Example keyfile with tracklab ids included:')
keyfiles.head()

Example keyfile with tracklab ids included:


,school,class,subject_id,person_id,consent,comment,sID_survey,ID_survey,source,date,tag_id,tracklab_id
0,45,105,1,NaN,1,NaN,20,2221,keyfile school 45 class 105,2023-06-08,22,0x24046130B6FA
1,45,105,2,NaN,1,NaN,21,2222,keyfile school 45 class 105,2023-06-08,20,0x24046130CCF9
2,45,105,3,NaN,1,NaN,22,2223,keyfile school 45 class 105,2023-06-08,11,0x24025F44F682
3,45,105,4,NaN,1,NaN,23,2224,keyfile school 45 class 105,2023-06-08,5,0x24046131F437
4,45,105,5,NaN,1,NaN,24,2225,keyfile school 45 class 105,2023-06-08,2,0x24025F48A133


# Survey scores
## Teacher responses

Teacher survey responses look like they were collected through an online form. Delivered raw as wide-format SPSS files.

School and class data has been added to each dataframe from source filename as a workaround because I was not able to determine which columns contained this data at first. 

Based on reading the codebook and inspecting the answers in the raw files, I've determined the following:

* Q30 = school
* Q31 = class
* Q32 = ?
* Q27 = T_gender
* Q28 = T_age
* Q29 = T_dutch
* Q30.0 = T_exp1
* Q31.0 = T_exp2
* Q32.0 = T_time_teaching
* Q33 = T_class_comp

In [367]:
# Determine path to raw SPSS files
folder = DATA_DIR / '01_survey' / 'teacher_raw_2023'

# Initiate empty dict to store teacher questionnaire dfs
tq_all = {}

print('Loading teacher responses...')
for file in folder.glob('*.sav'):
    var_name = file.stem
    df = pd.read_spss(file)
    
    # Add school/class as columns to each df from filename
    school_num, class_num = var_name.split('_')[1:]
    df['class'] = class_num
    
    # Change school 1 to 44
    if school_num != '1':
        df['school'] = school_num
    else:
        df['school'] = '44'
        
    # Store df in dict with filename as key
    tq_all[var_name] = df
    print(f'{var_name}')
    
print(f'Total dataframes: {len(tq_all)}')

# Uncomment to inspect example of available columns
# print(tq_all['tq_1_104'].columns.tolist())

Loading teacher responses...
tq_49_113
tq_1_104
tq_41_100
tq_41_101
tq_42_102
tq_43_103
tq_45_105
tq_45_106
tq_46_107
tq_47_108
tq_49_110
tq_49_111
tq_49_112
Total dataframes: 13


In [368]:
# Initiate empty dict to store relevant tq only
tq_relevant = {}

print('Finding teacher responses matching available tracking data...')
for df in tq_all:
    school_num = str(tq_all[df]['school'].iloc[0])
    class_num = str(tq_all[df]['class'].iloc[0])
    
    tq_match = tracking_summary[
        (tracking_summary['school'] == school_num) &
        (tracking_summary['class'] == class_num)
    ]
    
    if not tq_match.empty:
        tq_relevant[df] = tq_all[df]
        print(f"{df}")

#todo concatenate AFTER the columns have been equalized
# tq = pd.concat(tq_relevant, ignore_index=True)

print(f"Total matching: {len(tq_relevant)}")

# Uncomment to inspect dataset
# tq_relevant['tq_1_104']

Finding teacher responses matching available tracking data...
tq_1_104
tq_42_102
tq_43_103
tq_45_105
tq_45_106
tq_46_107
tq_47_108
Total matching: 7


### IOP scores

IOP response `Q68` is given per student. Variable name format is `Q68_N`, where N should match an entry in `keyfiles['subject ID']`. By matching the subject ID, `Q68` can then be matched to the 4-digit `ID` in the file containing student survey responses (once these have been fixed). For an initial analysis, the matching to the `subject ID` and thus to tracking tag numbers should be enough. 

Confirmed with Nathalie that IOP responses were optional. If IOP response was given, variable `Q68` is followed by Q70 and Q71 with matching student number. **We are only interested in Q68.**

* Q68 question: "In vergelijking met andere leerlingen bezoek ik [naam kind]"
* Q68 responses: Minder vaak, Gemiddeld, Vaker

**14-05-2025:** ⚠️ After multiple attempts, school 45-105 could not be matched for IOP scores since the student IDs cannot be properly matched. 

In [370]:
### Eliminate irrelevant columns in tq dataframes

# Lists of relevant questions
descriptives = ['Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33']
iop_id = ['Q68']  # Add 'Q70', 'Q71' for detailed IOP responses

tq_filtered = {}

for key, df in tq_relevant.items():
    # Create a mask for columns to keep
    cols_to_keep = []
    
    for col in df.columns:
        # Check if column matches any descriptive column
        if any(q_id in col for q_id in descriptives):
            cols_to_keep.append(col)
        # Check if column contains any of the specified question IDs
        elif any(q_id in col for q_id in iop_id):
            cols_to_keep.append(col)
    
    # Create a new dataframe with only the columns to keep
    tq_filtered[key] = df[cols_to_keep]


### Load IOP Q68 values into keyfiles df

# Create empty column 'iop' in keyfiles
if 'iop' not in keyfiles.columns:
    keyfiles['iop'] = None

# Iterate through each df in the dict
for df_name, df in tq_filtered.items():
    # Copy df to avoid pandas errors...
    df_copy = df.copy()
    
    # Extract valid school and class values
    school_num = df_copy['Q30'].unique().astype(str)
    class_num = df_copy['Q31'].unique().astype(str)
    
    for s, c in zip(school_num, class_num):
        if s in tracking_summary['school'].astype(str).unique():
            school_num = s
            class_num = c
            break
    else:
        school_num = np.nan
        class_num = np.nan
    
    # Identify Q68_N columns
    q68_cols = [col for col in df_copy.columns if col.startswith('Q68_')]
    
    # Iterate through df rows
    for idx, row in df_copy.iterrows():
        # school = row['Q30']
        # class_val = row['Q31']
        # 
        # Check each Q68_N column for matching subjects
        for q68_col in q68_cols:
            # Extract N from Q68_N column name
            subject_id = q68_col.split('_')[1]
            
            # Get value from this Q68_N cell
            q68_value = row[q68_col]
            
            # Only proceed if the cell has a valid value
            if pd.notna(q68_value) and str(q68_value) != "0" and str(q68_value) != "":
                # Find matching rows in keyfiles where all three conditions are met
                matching_rows = keyfiles[(keyfiles['school'] == school_num) & 
                                        (keyfiles['class'] == class_num) & 
                                        (keyfiles['sID_survey'] == subject_id)]
                
                # If matches found, update the 'iop' column with the actual value from Q68_N
                if not matching_rows.empty:
                    keyfiles.loc[matching_rows.index, 'iop'] = q68_value

### STRS scores

**14-05-2025:** I cannot find the STRS scores anywhere in the data. Requested help from Yvonne and Nathalie. 

In [379]:
temp = tq_relevant['tq_45_105']
# strs_cols = [col for col in temp.columns if col.str.contains('strs')]
# print(temp.columns.tolist())

## Student responses

### Aligning with keyfiles

**01-05-2025:** According to Nathalie, the order of the subject IDs ('subjectID') in the keyfiles are correct and can be matched to the student responses in the column 'sID'. The 4-digit person ID numbers ('id') were noted down incorrectly in the keyfiles. They can be copied from the student responses, according to 'sID'.

**08-05-2025:** After several hours spent trying to match student responses to keyfiles, it wasn't working. The data was matched **manually** with Maja copying student survey IDs into the separate keyfiles. 

In [381]:
# Load raw file
filename = 'TotalData_T1_all_cbs_ethnicity_gender'
path = DATA_DIR / '01_survey' / f"{filename}.xlsx"

students_raw = pd.read_excel(path)

students = students_raw.copy()

# Rename columns to match keyfiles
students = students.rename(columns={'School_ID': 'school', 'Class_ID': 'class', 'sID': 'subject_id', 'ID': 'person_id'}).astype(str)

# Change school '1' to '44' to match other datasets
students.loc[students['school']=='1', 'school'] = '44'

# Uncomment to inspect beginning of students df
# students.head()

### Inspect number of absent students
# Tried to see if it matches the amount of student data we have, but the variables turned out not to be filled in most of the time -> useless
# 
# temp = students
# temp['absent'] = temp['absent'].astype(int)
# temp.groupby(['school','class','nPupils','nAbsent'])['absent'].sum()

### Cleaning

In [382]:
# Count of all students per class 
s_count = pd.pivot_table(
    students,
    index=['school', 'class','nPupils'],
    values=['person_id'],
    aggfunc=pd.Series.nunique,
    margins = True,
    margins_name='Total'
)

print(f"Total students in raw dataset: {students['person_id'].nunique()}")
s_count

### INSPECT CONSENT VALUES
# # print(students['consent'].unique())  # Output: ['4' '1' '3' '5']
# students.loc[students['consent']!='1', 'consent'] = 'no'
# students.loc[students['consent']=='1', 'consent'] = 'yes'
# 
# # Count of students with(out) consent
# s_consent = pd.pivot_table(
#     students,
#     index=['school', 'class'],
#     columns=['consent'],
#     values='person_id',
#     aggfunc=pd.Series.nunique,
#     margins = True,
#     margins_name='Total'
# )
# 
# # s_consent['3'] = s_consent[['3', '4', '5']].sum()
# s_consent

Total students in raw dataset: 310


person_id
school class nPupils           
31     79    19              19
       80    22              22
32     73    24              24
       74    23              23
       75    26              26
41     100   25              25
       101   25              25
42     102   15              15
43     103   33              31
44     104   17              17
45     105   22              22
       106   21              21
46     107   21              21
47     108   19              19
Total                       310

In [383]:
students = students.loc[
    (students['school']).isin(tracking_data['school'].unique()) &
    (students['class']).isin(tracking_data['class'].unique())
]

print(f"Total students in dataset after removing schools/classes not available in tracking data: {students['person_id'].nunique()}")

# Count of all students per class 
s_count = pd.pivot_table(
    students,
    index=['school', 'class'],
    values=['person_id'],
    aggfunc=pd.Series.nunique,
    margins = True,
    margins_name='Total'
)

s_count

Total students in dataset after removing schools/classes not available in tracking data: 146


person_id
school class           
42     102           15
43     103           31
44     104           17
45     105           22
       106           21
46     107           21
47     108           19
Total               146

Students from 45/105 are all marked as `tracking = 0`, but all except one student were tracked. Variable is unreliable and ultimately not necessary - **ignore it.**

In [384]:
print(f"Total students tracked according to student master file: {students.loc[students['tracking']=='1', 'tracking'].count()}\n")

s_tracking = pd.pivot_table(
    students,
    index=['school', 'class','nPupils'],
    columns=['tracking'],
    values='person_id',
    aggfunc=pd.Series.nunique,
    margins = True,
    margins_name='Total'
).fillna(0)

s_tracking

Total students tracked according to student master file: 124



tracking                 0      1  Total
school class nPupils                    
42     102   15        0.0   15.0     15
43     103   33        0.0   31.0     31
44     104   17        0.0   17.0     17
45     105   22       22.0    0.0     22
       106   21        0.0   21.0     21
46     107   21        0.0   21.0     21
47     108   19        0.0   19.0     19
Total                 22.0  124.0    146

In [385]:
# Count the kids in keyfiles ID_survey equals ID in students
# Where tagnumber != 1111 or 35

keyfile_filtered = keyfiles
keyfile_filtered.loc[keyfile_filtered['school']=='44', 'school'] = '1'
keyfile_filtered = keyfile_filtered.loc[
    (keyfile_filtered['school'].isin(tracking_data['school'].unique())) &
    (keyfile_filtered['class'].isin(tracking_data['class'].unique())) &
    (keyfile_filtered['ID_survey'].isin(students['person_id'].unique())) &
    (keyfile_filtered['tag_id'] != '9999') &
    (keyfile_filtered['subject_id'] != '9999')
    ]

s_trackers = pd.pivot_table(
    keyfile_filtered,
    index=['school', 'class'],
    values=['ID_survey', 'tag_id', 'tracklab_id'],
    aggfunc = pd.Series.nunique,
)

s_trackers.loc['Total'] = s_trackers.sum()

# temp.loc[temp[['school','class','tracklab_id']].duplicated(keep=False)] -> NO TRACKLAB IDS ARE DUPLICATED PER CLASS

# Limit student survey df to entries that have matching tracking tags
students = students.loc[students['person_id'].astype(str).isin(keyfile_filtered['ID_survey'].astype(str))]
# students['person_id'].nunique()

# Display data
print(f"Total students whose tracking data can be paired with survey: {s_trackers.iloc[-1]['tag_id']}\n")
s_trackers

Total students whose tracking data can be paired with survey: 115



,ID_survey,tag_id,tracklab_id
"(42, 102)",14,14,14
"(43, 103)",28,28,28
"(45, 105)",21,21,21
"(45, 106)",15,15,15
"(46, 107)",18,18,18
"(47, 108)",19,19,19
Total,115,115,115


### SPARTS scores

Source:  https://doi.org/10.1111/bjep.12094

Relevant variables named 'SPARTSN' (e.g. 'SPARTS1') in the codebook, but this name is not present in the data. Instead, variables named **'st_relN'** have been identified as SPARTS scores. As explained in the codebook, the questionnaire contained 13 items, but Q13 was not presented to all students. After filtering the dataset for relevant data only (i.e., responses of students whose tracking data we have available), only responses 1-12 were available anyway.  

Q12 is not part of the original scale, but developed for this study.

I cannot find a score sheet for this test that is not behind a paywall. The COTAN entry for the SPARTS lists a 25-item test instead of the 13-item test used. 

**08-05-2025:** Informed by Nathalie that no scoring sheet exists. Will separate SPARTS scores and compute raw scores. 

In [386]:
# Create the new dataframe with person_id and columns containing 'st_rel'
sparts_cols = [col for col in students.columns if 'st_rel' in col]
sparts = students[['person_id'] + sparts_cols]
sparts = sparts[sorted(sparts.columns)]
print(f"Total students with SPARTS entries: {sparts['person_id'].nunique()}\n")

# Drop rows containing any NaNs in SPARTS scores
sparts[sparts_cols] = sparts[sparts_cols].replace('nan', np.nan)
sparts = sparts.dropna(subset=sparts_cols, how='any')
sparts[sparts_cols] = sparts[sparts_cols].astype(float).astype(int)
print(f"Total students after removing entries with incomplete SPARTS responses: {sparts['person_id'].nunique()}\n")

# Rename st_rel columns to sparts
sparts = sparts.rename(columns={col: col.replace('st_rel', 'SPARTS') for col in sparts.columns if 'st_rel' in col})

### CALCULATE SPARTS SCORES
closeness_cols = ['SPARTS1', 'SPARTS2', 'SPARTS3', 'SPARTS4', 'SPARTS5', 'SPARTS13']
conflict_cols = ['SPARTS6', 'SPARTS7', 'SPARTS8', 'SPARTS9', 'SPARTS10', 'SPARTS11']
vos_col = 'SPARTS12'

sparts['SPARTS_closeness'] = sparts[closeness_cols].sum(axis=1)
sparts['SPARTS_conflict'] = sparts[conflict_cols].sum(axis=1)

# Reverse code conflict_cols to compute SPARTS_total
for col in conflict_cols:
    reversed_col = col + '_R'
    # Reverse coding: 1→5, 2→4, 3→3, 4→2, 5→1
    sparts[reversed_col] = 6 - sparts[col]

# Create list of reversed conflict column names
conflict_reversed = [col + '_R' for col in conflict_cols]

# Calculate total score (sum of closeness items and reversed conflict items)
sparts['SPARTS_total'] = sparts[closeness_cols + conflict_reversed].sum(axis=1)

###### FILTER KEYFILE 

# Attach SPARTS scores to keyfile
keyfile_filtered = keyfile_filtered.merge(
    sparts,
    left_on='ID_survey',
    right_on='person_id',
    how='right',
    indicator=True
).drop('person_id_y', axis=1).rename(columns={'person_id_x': 'person_id'})

print('Example keyfile after merging SPARTS responses:')
display(keyfile_filtered.head())

Total students with SPARTS entries: 115

Total students after removing entries with incomplete SPARTS responses: 107

Example keyfile after merging SPARTS responses:


,school,class,subject_id,person_id,consent,comment,sID_survey,ID_survey,source,date,...,SPARTS_closeness,SPARTS_conflict,SPARTS6_R,SPARTS7_R,SPARTS8_R,SPARTS9_R,SPARTS10_R,SPARTS11_R,SPARTS_total,_merge
0,42,102,1,2107,1,NaN,1,2158,keyfile school 42 class 102,2023-04-11,...,25,6,5,5,5,5,5,5,55,both
1,42,102,3,2109,1,NaN,3,2161,keyfile school 42 class 102,2023-04-11,...,10,19,1,4,3,3,1,5,27,both
2,42,102,4,2110,1,NaN,4,2162,keyfile school 42 class 102,2023-04-11,...,23,7,5,5,5,5,5,4,52,both
3,42,102,5,2111,1,NaN,5,2163,keyfile school 42 class 102,2023-04-11,...,18,14,4,4,3,4,3,4,40,both
4,42,102,6,2112,1,NaN,6,2164,keyfile school 42 class 102,2023-04-11,...,25,12,4,4,3,5,4,4,49,both


# Final dataset

In [ ]:
master = keyfile_filtered.copy()

## N participants

In [ ]:
### SCHOOLS & CLASSES
master.groupby('school')['class'].nunique()

#todo N students
#todo N teachers

## Age

In [ ]:
#todo teacher age

### STUDENT AGE
master['age'].describe()

### Gender

I tried to match student gender information to the keyfile, but it turns out that **gender information for 2023 entries is not available**. 

In [387]:
# Attach 'gender' column from student survey responses to keyfile
keyfile_filtered = keyfile_filtered.merge(
    students[['person_id', 'Gender', 'age']],
    left_on='ID_survey',
    right_on='person_id',
    how='left'
)

keyfile_filtered['age'] = keyfile_filtered['age'].astype(float).astype(int)

print(f"Unique entries in column 'gender': {keyfile_filtered['Gender'].unique()}")

Unique entries in column 'gender': ['nan']


### Cleaning consent (again)

Student `ID_survey = 2283` had only oral parental consent, so I originally marked them as 9 (no consent). However, their tracking data and SPARTS scores are present, so I will change the entry to 1 (consent). This student is missing corresponding IOP scores, so only a one-way student-teacher relationship can be examined. 

In [388]:
print('Entries with comments - printed for manual inspection:')
display(keyfile_filtered.loc[~keyfile_filtered['comment'].isna()])

print('Entries with consent != 1 - printed for manual inspection:')
display(keyfile_filtered.loc[keyfile_filtered['consent']!='1'])

keyfile_filtered.loc[keyfile_filtered['ID_survey']=='2283', 'consent'] = '1'

Entries with comments - printed for manual inspection:


,school,class,subject_id,person_id_x,consent,comment,sID_survey,ID_survey,source,date,...,SPARTS7_R,SPARTS8_R,SPARTS9_R,SPARTS10_R,SPARTS11_R,SPARTS_total,_merge,person_id_y,Gender,age
20,43,103,10,2181,1,Vult vragenlijst niet in,10,2183,keyfile school 43 class 103,2023-05-11,...,5,5,5,5,5,58,both,2183,nan,12
48,45,105,10,NaN,1,it could be that not tag 12 but 30 was used fo...,29,2230,keyfile school 45 class 105,2023-06-08,...,3,1,5,3,4,47,both,2230,nan,11
86,46,107,20,2222,9999,"since student had no written consent, their na...",20,2283,keyfile school 46 class 107,2023-05-23,...,5,2,5,5,5,52,both,2283,nan,12


Entries with consent != 1 - printed for manual inspection:


,school,class,subject_id,person_id_x,consent,comment,sID_survey,ID_survey,source,date,...,SPARTS7_R,SPARTS8_R,SPARTS9_R,SPARTS10_R,SPARTS11_R,SPARTS_total,_merge,person_id_y,Gender,age
86,46,107,20,2222,9999,"since student had no written consent, their na...",20,2283,keyfile school 46 class 107,2023-05-23,...,5,2,5,5,5,52,both,2283,nan,12


## Export

Master file is exported to *workgroup folder > dsp > data > 02_interim*

In [348]:
filename = 'dataset_partial'
today = pd.to_datetime('today').strftime('%Y-%m-%d_%H-%M')
savepath = DATA_DIR / '02_interim' / f"{filename}_{today}.xlsx"
master.to_excel(savepath, index=False, engine='openpyxl')

### OLD: Export merged file

**08-05-2025:** Below is the code used by Maja to export the first version of the keyfile. Not used anymore, but will keep it here commented.

File includes connection school & class -> subject_id -> tagnumber -> tracklab_id + iop

In [ ]:
# ### Temporarily rename school 1 to school 44 to match data
# temp = tracking_data.copy().astype(str)
# temp.loc[temp['school']=='1', 'school'] = '44'
# 
# # temp[temp['school']=='44']
# #todo find out why school 44 has been (inconsistently) renamed school 1, and how we should call it
# 
# # Find the missing tracklab_ids for each school-class combination
# missing_entries = []
# 
# # Get unique school-class combinations from keyfiles
# school_class_combinations = keyfiles[['school', 'class']].drop_duplicates()
# 
# # For each school-class combination
# for _, row in school_class_combinations.iterrows():
#     school = row['school']
#     class_val = row['class']
#     
#     # Get all tracklab_ids for this school-class in tracking_data
#     tracking_ids = temp[(temp['school'] == school) & 
#                         (temp['class'] == class_val)]['tracklab_id'].unique()
#     
#     # Get all tracklab_ids for this school-class already in keyfiles
#     keyfiles_ids = keyfiles[(keyfiles['school'] == school) & 
#                            (keyfiles['class'] == class_val)]['tracklab_id'].unique()
#     
#     # Find tracklab_ids in tracking_data but not in keyfiles
#     missing_ids = set(tracking_ids) - set(keyfiles_ids)
#     
#     # Create new rows for each missing tracklab_id
#     for missing_id in missing_ids:
#         # Create a new row with school, class, and tracklab_id
#         new_row = {
#             'school': school,
#             'class': class_val,
#             'tracklab_id': missing_id
#         }
#         missing_entries.append(new_row)
# 
# # Create df from the missing entries
# if missing_entries:
#     missing_df = pd.DataFrame(missing_entries)
#     
#     # Append missing entries to keyfiles
#     keyfiles = pd.concat([keyfiles, missing_df], ignore_index=True)
#     
#     print(f"Added {len(missing_entries)} new rows to keyfiles for missing tracklab_ids")
# else:
#     print("No missing tracklab_ids found.")
#     
# ### EXPORT TO EXCEL
# export = keyfiles.loc[keyfiles['school'].isin(temp['school'].unique())]
# 
# filename = 'merged-data-WIP'
# today = pd.to_datetime('today').strftime('%Y-%m-%d_%H-%M')
# savepath = DATA_DIR / '02_interim' / f"{filename}_{today}.xlsx"
# # export.to_excel(savepath, index=False, engine='openpyxl')
# 
# # Display df
# keyfiles.sort_values(by=['school', 'class', 'tracklab_id'])[['school', 'class', 'tracklab_id']]

In [ ]:
# sorting_columns = ['subject_id', 'ID_survey', 'tagnumber']
# keyfiles.loc[keyfiles['tagnumber']=='-', 'tagnumber'] = 1111
# keyfiles.loc[~keyfiles['tagnumber'].astype(str).str.isdigit(), 'tagnumber'] = 1111
# # keyfiles.loc[~keyfiles['consent'].astype(str).str.isdigit(), 'consent'] = 5
# keyfiles[sorting_columns] = keyfiles[sorting_columns].fillna(1111).astype(int, errors='raise') 
# keyfiles.sort_values(by=['school', 'class', 'subject_id','consent','tagnumber'])